<a href="https://colab.research.google.com/github/alligmckernan/ADS-505-Final-Project/blob/main/ADS_505_Team_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =====================================================
# ADS-505 Final Project — Miscarriage Prediction
# =====================================================

# 1. Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# Import CalibrationDisplay after upgrading scikit-learn
from sklearn.calibration import CalibrationDisplay


import warnings
warnings.filterwarnings("ignore")

# Random seed
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Define the path to the dataset
# The file is located directly in the /content directory, not in Google Drive.
file_path = '/content/Miscarriage_Prediction_dataset_New_HA.csv'
# Load the dataset

# Changed the delimiter to semicolon
df = pd.read_csv(file_path, delimiter=';')
print("Shape:", df.shape)

# Convert 'BMI' column to numeric after replacing commas with periods
df['BMI'] = df['BMI'].str.replace(',', '.', regex=False).astype(float)

# Display the first few rows and the columns and their data types
display(df.head())
display(df.info())

In [ ]:
# =====================================================
# Data overview
# =====================================================
print(df.info())
print(df.describe(include='all').T)

# Missing values
missing = df.isna().mean().sort_values(ascending=False)
print("\nMissing values (top 10):")
print(missing.head(10))

In [ ]:
# =====================================================
# Exploratory Data Analysis (per Shmueli et al., 2020 Ch. 3)
# =====================================================
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from scipy.stats import mannwhitneyu

sns.set(style="whitegrid")

TARGET_COL = 'Miscarriage/ No Miscarriage'

# --- 1. Basic structure and summary ---
print("\nData structure:")
print(df.shape)
print(df.info())
print("\nSummary statistics:")
display(df.describe(include='all').T)

# --- 2. Correlation analysis (numeric variables only) ---
corr_matrix = df.corr(numeric_only=True)
corr_with_target = corr_matrix[TARGET_COL].sort_values(ascending=False)
print("\nCorrelation of numeric variables with target:")
print(corr_with_target.head(10))

# --- 3. Select top correlated predictors (top 8 excluding target) ---
top_corrs = corr_with_target.index[1:9]
print("\nTop correlated predictors:", list(top_corrs))

# --- 4. Histograms (Lab 1 style visual check) ---
for col in top_corrs:
    plt.figure(figsize=(6,3))
    sns.histplot(df, x=col, hue=TARGET_COL, bins=30, kde=True, palette='muted')
    plt.title(f"Distribution of {col} by Miscarriage Outcome")
    plt.xlabel(col); plt.ylabel("Count")
    plt.tight_layout(); plt.show()

# --- 5. Boxplots for group comparison (Shmueli et al. §3.4) ---
for col in top_corrs:
    plt.figure(figsize=(6,3))
    sns.boxplot(x=TARGET_COL, y=col, data=df, palette='Set2')
    plt.title(f"{col} vs Miscarriage Outcome")
    plt.xlabel("Outcome (0 = No Miscarriage, 1 = Miscarriage)")
    plt.ylabel(col)
    plt.tight_layout(); plt.show()

# --- 6. Non-parametric test for mean/median differences (as in Labs) ---
print("\nMann–Whitney U Tests for Group Differences")
print("---------------------------------------------------")
for col in top_corrs:
    group0 = df.loc[df[TARGET_COL]==0, col].dropna()
    group1 = df.loc[df[TARGET_COL]==1, col].dropna()
    stat, p = mannwhitneyu(group0, group1, alternative='two-sided')
    signif = "Significant" if p < 0.05 else "ns"
    print(f"{col:<20}  U = {stat:>10.1f}  p = {p:>.4f}  → {signif}")

# --- 7. Correlation heatmap (Lab visualization) ---
plt.figure(figsize=(8,6))
sns.heatmap(df[top_corrs.tolist() + [TARGET_COL]].corr(), cmap='coolwarm', center=0, annot=False)
plt.title("Correlation Matrix — Top Predictors and Target")
plt.tight_layout(); plt.show()


In [ ]:
# =====================================================
# DATA PREPROCESSING — Train/Test Split Only
# =====================================================

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Define target column
TARGET_COL = 'Miscarriage/ No Miscarriage'  # <-- change to actual target name
if TARGET_COL not in df.columns:
    raise ValueError(f"Target column '{TARGET_COL}' not found. Available: {df.columns.tolist()}")

# Separate features (X) and target (y)
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL].astype(int)

# Identify numeric and categorical columns
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]
print("Numeric columns:", num_cols)
print("Categorical columns:", cat_cols)

# Train/Test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

# Pipelines for preprocessing
numeric_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combine preprocessing
preprocess = ColumnTransformer([
    ('num', numeric_pipe, num_cols),
    ('cat', categorical_pipe, cat_cols)
])

# Fit the preprocessor on train data only
preprocess.fit(X_train)

# Transform both sets
X_train_prepared = preprocess.transform(X_train)
X_test_prepared = preprocess.transform(X_test)

print("Preprocessing complete.")
print("Transformed train shape:", X_train_prepared.shape)
print("Transformed test shape:", X_test_prepared.shape)


In [ ]:
# =====================================================
# MODELS — Logistic Regression & Random Forest (Train/Test only)
# =====================================================
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    roc_auc_score, classification_report, confusion_matrix,
    RocCurveDisplay, PrecisionRecallDisplay
)
from sklearn.calibration import calibration_curve

RANDOM_STATE = 42

def evaluate_classifier(name, clf, X_tr, y_tr, X_te, y_te):
    clf.fit(X_tr, y_tr)

    # Probabilities & predictions
    proba = clf.predict_proba(X_te)[:, 1]
    preds = (proba >= 0.5).astype(int)

    # Metrics
    auc = roc_auc_score(y_te, proba)
    print(f"\n=== {name} — Test Set ===")
    print("ROC-AUC:", round(auc, 4))
    print("Confusion Matrix:\n", confusion_matrix(y_te, preds))
    print(classification_report(y_te, preds, digits=4))

    # Curves
    RocCurveDisplay.from_predictions(y_te, proba)
    plt.title(f'ROC Curve — {name}')
    plt.show()

    PrecisionRecallDisplay.from_predictions(y_te, proba)
    plt.title(f'Precision-Recall — {name}')
    plt.show()

    # Simple calibration curve (version-safe)
    prob_true, prob_pred = calibration_curve(y_te, proba, n_bins=10, strategy='uniform')
    plt.plot(prob_pred, prob_true, marker='o')
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.xlabel('Predicted probability')
    plt.ylabel('Observed frequency')
    plt.title(f'Calibration — {name}')
    plt.show()

    return auc, proba, preds, clf


# ----------------- 1) Logistic Regression -----------------
log_reg = LogisticRegression(
    max_iter=2000,
    solver='saga',
    random_state=RANDOM_STATE
)
auc_lr, proba_lr, preds_lr, clf_lr = evaluate_classifier(
    "Logistic Regression", log_reg, X_train_prepared, y_train, X_test_prepared, y_test
)


# ----------------- 2) Random Forest -----------------
rf = RandomForestClassifier(
    n_estimators=300,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
auc_rf, proba_rf, preds_rf, clf_rf = evaluate_classifier(
    "Random Forest", rf, X_train_prepared, y_train, X_test_prepared, y_test
)

print("\nAUC comparison:", {"LogReg_AUC": round(auc_lr, 4), "RF_AUC": round(auc_rf, 4)})


# =====================================================
# FEATURE IMPORTANCE (Random Forest) with proper names
# =====================================================
# Build feature names from the fitted preprocessor

num_names = preprocess.named_transformers_['num'].get_feature_names_out(num_cols) \
            if hasattr(preprocess.named_transformers_['num'], 'get_feature_names_out') else np.array(num_cols)

ohe = preprocess.named_transformers_['cat'].named_steps['encoder'] if 'cat' in preprocess.named_transformers_ else None
if ohe is not None and len(cat_cols) > 0:
    cat_names = ohe.get_feature_names_out(cat_cols)
    feature_names = np.concatenate([num_names, cat_names])
else:
    feature_names = np.array(num_names)

# Show top 20 importances
if hasattr(clf_rf, "feature_importances_"):
    importances = clf_rf.feature_importances_
    order = np.argsort(importances)[::-1][:20]
    plt.barh(feature_names[order][::-1], importances[order][::-1])
    plt.title("Top 20 Feature Importances — Random Forest")
    plt.tight_layout()
    plt.show()
else:
    print("Feature importances not available for this model.")




In [ ]:
# =====================================================
# 3) Support Vector Machine (RBF)
# =====================================================
from sklearn.svm import SVC

svm_rbf = SVC(
    kernel='rbf',
    C=1.0,               # regularization
    gamma='scale',       # RBF bandwidth (auto scaling)
    probability=True,    # needed for predict_proba and ROC/PR curves
    random_state=42
)

auc_svm, proba_svm, preds_svm, clf_svm = evaluate_classifier(
    "SVM (RBF)", svm_rbf, X_train_prepared, y_train, X_test_prepared, y_test
)

print("\nAUC comparison (updated):",
      {"LogReg_AUC": round(auc_lr, 4),
       "RF_AUC": round(auc_rf, 4),
       "SVM_RBF_AUC": round(auc_svm, 4)})

